# Digit Recognition

In [ ]:
# Only run once, to download MNIST.

import urllib.request
import os

# Create an 'mnist' directory unless it exists:
LOCAL_DIR = './mnist/'
if not os.path.exists(LOCAL_DIR):
    os.makedirs(LOCAL_DIR)

# Download the four MNIST files from the official site:
MNIST_SITE = 'http://yann.lecun.com/exdb/mnist/'
TRAINING_IMAGES = 'train-images-idx3-ubyte.gz'
TRAINING_LABELS = 'train-labels-idx1-ubyte.gz'
TEST_IMAGES = 't10k-images-idx3-ubyte.gz'
TEST_LABELS = 't10k-labels-idx1-ubyte.gz'

urllib.request.urlretrieve(MNIST_SITE + TRAINING_IMAGES, LOCAL_DIR + TRAINING_IMAGES)
urllib.request.urlretrieve(MNIST_SITE + TRAINING_LABELS, LOCAL_DIR + TRAINING_LABELS)
urllib.request.urlretrieve(MNIST_SITE + TEST_IMAGES, LOCAL_DIR + TEST_IMAGES)
urllib.request.urlretrieve(MNIST_SITE + TEST_LABELS, LOCAL_DIR + TEST_LABELS)

print("Data loaded")